In [17]:
import requests 
import pandas as pd

In [18]:
import requests
import pandas as pd
import re

url = 'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2007-01-01T00:00&end_date=2007-12-31T23:59&time_trunc=month'

def llamar_api(url):

    llamada = requests.get(url)  

    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  

    if llamada.status_code != 200:  
        print(f"El motivo por el que la llamada falló es {llamada.reason}")  
    else:
        return llamada.json()



In [19]:
json_energia= llamar_api(url)



La llamada a la API nos ha dado una respuesta de tipo: 200


In [20]:
json_energia.keys()

dict_keys(['data', 'included'])

In [22]:
json_energia['included']
#type(json_energia['included'])



[{'type': 'Hidráulica',
  'id': '10288',
  'groupId': '1',
  'attributes': {'title': 'Hidráulica',
   'description': None,
   'color': '#0090d1',
   'type': 'Renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2019-08-21T19:30:58.000+02:00',
   'values': [{'value': 2265540,
     'percentage': 0.08811213468383587,
     'datetime': '2007-01-01T00:00:00.000+01:00'},
    {'value': 3018536,
     'percentage': 0.13154166297706543,
     'datetime': '2007-02-01T00:00:00.000+01:00'},
    {'value': 3928392,
     'percentage': 0.16071782970429438,
     'datetime': '2007-03-01T00:00:00.000+01:00'},
    {'value': 2867464,
     'percentage': 0.13036017775941525,
     'datetime': '2007-04-01T00:00:00.000+02:00'},
    {'value': 2837474,
     'percentage': 0.12333368685692031,
     'datetime': '2007-05-01T00:00:00.000+02:00'},
    {'value': 2309410,
     'percentage': 0.10057546914220436,
     'datetime': '2007-06-01T00:00:00.000+02:00'},
    {'value': 2454750,
     'percentage'

In [24]:
# Iniciar una función que toma dos argumentos: energia_especifica y diccionario_resultados.
def extraer_informacion_energia(energia_especifica, diccionario_resultados):
    
    # Iniciamos un bucle for que recorre una lista de elementos en la clave "values" de energia_especifica. Recordemos que energía especifica será cada uno de los diccionarios que teníamos en el json que devuelve la API. 
    # Su longitud es de 17
    for mes in energia_especifica["attributes"]["values"]:
        
        # Agregamos el valor de la clave "datetime" del elemento actual mes a la lista asociada con la clave "fecha" en diccionario_resultados.
        diccionario_resultados["fecha"].append(mes["datetime"])
        
        # Agregamos el valor de la clave "value" del elemento actual mes a la lista asociada con la clave "valor" en diccionario_resultados.       
        diccionario_resultados["valor"].append(mes["value"])
        
        # Agregamos el valor de la clave "title" de la clave "attributes" en el objeto energia a la lista asociada con la clave "tipo_energia" en diccionario_resultados.
        diccionario_resultados["tipo_energia"].append(energia["attributes"]["title"])
        
        # Agregamos el valor de la clave "type" de la clave "attributes" en el objeto energia a la lista asociada con la clave "tipo_generacion" en diccionario_resultados.
        diccionario_resultados["tipo_generacion"].append(energia["attributes"]["type"])
    
    # Devuelve el diccionario actualizado diccionario_resultados.
    return diccionario_resultados


In [25]:

# Se crea un diccionario llamado diccionario_final con cuatro claves, y cada clave se asocia a una lista vacía.
diccionario_final = {"tipo_energia": [], "tipo_generacion": [], "fecha":[], "valor": []}

# Se inicia un bucle for que recorre los años desde 2011 hasta 2022 (inclusive).
for año in range(2011, 2023):

    # Se construye una URL utilizando el año actual en el bucle para consultar datos de generación de energía específicos para ese año.
    url = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=month"

    # Se realiza una solicitud HTTP GET a la URL para obtener los datos.
    response = requests.get(url)

    # Se imprime el código de estado de la respuesta HTTP, que indica si la solicitud fue exitosa (código 200) u otro resultado.
    print(response.status_code)

    # Se analiza la respuesta JSON obtenida de la solicitud.
    response_json = response.json()

    # Se extrae la lista de datos de generación de energía específica de la respuesta JSON.
    lista_energias = response_json["included"]

    # Se inicia un bucle for que recorre la lista de datos de generación de energía.
    for energia in lista_energias:

        # En cada iteración del bucle, se llama a la función extraer_informacion_energia con el objeto "energia" y el diccionario "diccionario_final" como argumentos.
        # La función agrega información específica de "energia" al diccionario "diccionario_final" y devuelve el diccionario actualizado, que se almacena nuevamente en "diccionario_final".
        diccionario_final = extraer_informacion_energia(energia, diccionario_final)


200
200
200
200
200
200
200
200
200
200
200
200


In [26]:
# convertir a DataFrame
df_energia = pd.DataFrame(diccionario_final)
df_energia.head()

,tipo_energia,tipo_generacion,fecha,valor
0,Hidráulica,Renovable,2011-01-01T00:00:00.000+01:00,5119512.380
1,Hidráulica,Renovable,2011-02-01T00:00:00.000+01:00,2998051.279
2,Hidráulica,Renovable,2011-03-01T00:00:00.000+01:00,3910363.276
3,Hidráulica,Renovable,2011-04-01T00:00:00.000+02:00,3537937.995
4,Hidráulica,Renovable,2011-05-01T00:00:00.000+02:00,2797500.027


In [27]:
df_energia.tail()

,tipo_energia,tipo_generacion,fecha,valor
2482,Generación total,Generación total,2022-08-01T00:00:00.000+02:00,2.528779e+07
2483,Generación total,Generación total,2022-09-01T00:00:00.000+02:00,2.366389e+07
2484,Generación total,Generación total,2022-10-01T00:00:00.000+02:00,2.195674e+07
2485,Generación total,Generación total,2022-11-01T00:00:00.000+01:00,2.157244e+07
2486,Generación total,Generación total,2022-12-01T00:00:00.000+01:00,2.222615e+07
